In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import secrets;
from sklearn.svm import SVC
from flask import render_template, request, redirect, flash
import pickle


In [2]:
def finalList(rank1,perc,category,state,gender,pwd,sortby):
    
    import os
    #base_path = Path(__file__).parent
    base_path = os.getcwd()
    file_path = 'round1_cleaned.csv'
    df = pd.read_csv(file_path)

    # The algorithm showed some anomaly when %tile was 100.
    # Hence the following condition.

    if(rank1 == '-1'):
        rank = float(pvr(perc,pwd,category))
    else:
        rank = rank1

    if(pwd == 'YES'):
        if(gender == 'M'):
            catg = category+'-PwD'
            p = df[(df['Closing Rank']>=rank)&((df['Category']==catg)|(df['Category']=='OPEN-PwD'))&(df['Seat Pool']=='Gender-Neutral')]
        else:
            catg = category+'-PwD'
            p = df[(df['Closing Rank']>=rank)&((df['Category']==catg)|(df['Category']=='OPEN-PwD'))]
    else:
        if(gender == 'M'):
            p = df[(df['Closing Rank']>=rank)&((df['Category']==category)|(df['Category']=='OPEN'))&(df['Seat Pool']=='Gender-Neutral')]
        else:
             p = df[(df['Closing Rank']>=rank)&((df['Category']==category)|(df['Category']=='OPEN'))]

    v = []
    for i in p.index:
        if(p['State'][i] == state):
            if(p['Quota'][i] == 'OS'):
                v.append(i)
        elif((p['State'][i]!='All India')&(p['State'][i]!=state)):
            if(p['Quota'][i] == 'HS'):
                v.append(i)

    q = p.drop(index = v)
    q = q.sort_values(sortby)
    x = q.drop(['Quota','Category','Seat Pool','Opening Rank','Closing Rank','State','Round No'],axis=1).drop_duplicates()
    x.reset_index(inplace = True, drop = True)
    return x


In [3]:
rvp=pd.read_csv('rvp_cleaned.csv')
#print('Create regressors', flush=True)

def create_regressor(rvp_):
    X=rvp_['PERCENTILE'].values.reshape(-1,1)
    y=rvp_['RANK'].values.reshape(-1,1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    return regressor

categories = ['SC']
regressors = {
        category : [
            create_regressor(rvp[rvp['CATEGORY']==category]),create_regressor(rvp[rvp['CATEGORY']==category + '-PwD']) ] for category in categories}

X=rvp['PERCENTILE'].values.reshape(-1,1)
y=rvp['RANK'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf = SVC()
clf.fit(X_train,y_train)
pickle.dump(clf,open('model.pkl','wb'))


def pvr(perc,pwd,category):
    x=pd.Series([perc])
    z=regressors[category][pwd=='YES'].predict(x.values.reshape(-1,1))
    k=float(np.round(z))
    if(k<=0):
        k=15
    return k


C:\Users\Anup\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
from flask import Flask
from flask_ngrok import run_with_ngrok

model=pickle.load(open('model.pkl','rb'))

app = Flask(__name__)


@app.route("/", methods=["GET","POST"])
def index():

    secret_key=secrets.token_hex(16)
    app.config["SECRET_KEY"] = secret_key

    if(request.method == "POST"):
        req = request.form

        percentile = req["percentile"]
        rank = req["rank"]
        state = req["state"]
        pwd = req["pwd"]
        gender = req["gender"]
        category = req["category"]
        sortby = str(req["sortby"])

        if(percentile == "" and rank == ""):
            flash("Please enter either your Rank or your Percentile",'error')
            return redirect(request.url)

        if(rank == ""):
            ranks = pvr(float(percentile),pwd,category);
            ranks = int(ranks);

            if(ranks <= 0):
                ranks = 2;
            result = finalList(ranks,float(percentile),category,state,gender,pwd,sortby);

        if(rank):
            result = finalList(int(rank),percentile,category,state,gender,pwd,sortby);
            ranks = rank;

        return render_template("public/result.html",ranks=ranks,category=category,tables=[result.to_html(classes='data')], titles=result.columns.values)

    return render_template("public/index.html")


In [5]:
if __name__ == '__main__':
    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2022 00:08:09] "POST / HTTP/1.1" 200 -
